In [ ]:
import json
import numpy as np
import re
import fastavro

In [ ]:
def convert_ascii_sextractor_catalog_to_json(data_filename,params_filename):
    with open(params_filename, 'r') as file:
        i = 0
        idx = {}
        for line in file:
            param = line.strip()
            param = param.replace("(","_")
            param = param.replace(")","")
            idx[param] = i
            i += 1
            
    with open(data_filename, 'r') as file:
        r = {'SExtractorSourceArray':[]}
        for line in file:
            json_dict = {}
            string_match = re.match(r"^#(.+)", line)

            try:
                # See if line is a comment, and skip if so.
                c = string_match.group(1)
                continue
            except:
                pass

            all = line.strip().split()

            for p in idx.keys():
                j = idx[p]
                json_dict[p] = all[j]
            r['SExtractorSourceArray'].append(json_dict)
    return r

In [ ]:
json_data = convert_ascii_sextractor_catalog_to_json('awaicgen_output_mosaic_image_resampled_refgainmatchsexcat.txt','../git/rapid/cdf/rapidSexParamsGainMatch.inp')

In [ ]:
json_data

In [ ]:
json_data_single = {"alertId": "001",
                    "SExtractorSourceSingle": json_data['SExtractorSourceArray'][0]}

In [ ]:
json_data_combined = {"alertId": "001",
                    "SExtractorSourceSingle": json_data['SExtractorSourceArray'][0],
                     "SExtractorSourceArray": json_data['SExtractorSourceArray']}

In [ ]:
sextractor_schema = {
    "namespace": "rapid",
	"name": "SExtractorSourceSingle",
	"doc": "RAPID avro alert schema",
    "version": "X.Y",
    "type": "record",
	"fields": []
}

In [ ]:
params_filename = '../git/rapid/cdf/rapidSexParamsGainMatch.inp'
with open(params_filename, 'r') as file:
        i=0
        idx = {}
        for line in file:
            param = line.strip()
            param = param.replace("(","_")
            param = param.replace(")","")
            idx[param] = i
            i+=1
        for p in idx.keys():
            sextractor_schema["fields"].append({"name": p, "type": ["null","string"],"default": None})

In [ ]:
sextractor_schema

In [ ]:
with open('SExtractorSourceSingle.avsc','w') as file:
    json.dump(sextractor_schema,file,indent=4)

In [ ]:
schema_definition = fastavro.schema.load_schema_ordered(['SExtractorSourceSingle.avsc','alert.avsc'])

In [ ]:
with open('sextractor_combined_alert.avro','wb') as avro_file:
    fastavro.writer(avro_file,schema_definition,[json_data_combined])

In [ ]:
with open('sextractor_single_alert.avro','rb') as avro_file:
    avro_records = fastavro.reader(avro_file)
    for record in avro_records:
        print(record)

In [ ]:
with open('sextractor_alert.avro','rb') as avro_file:
    avro_records = fastavro.reader(avro_file)
    for record in avro_records:
        print(record)

In [ ]:
with open('sextractor_combined_alert.avro','rb') as avro_file:
    avro_records = fastavro.reader(avro_file)
    for record in avro_records:
        print(record)